# Celery: 분산 작업 큐

![](https://d1q6f0aelx0por.cloudfront.net/product-logos/b0b66785-c2db-4cf4-94b0-de0240633060-celery.png)

## Overview

분산 Task Queue

+ 웹사이트 : [http://celeryproject.org/](http://celeryproject.org/)
+ [python package index](https://pypi.python.org/pypi/celery)
+ [Github 저장소](https://github.com/celery/celery/)

수행이 오래 걸리는 작업을 직접 처리하지 않고 별도의 작업자(Worker)를 두어, 수행할 작업을 위임하는 방식을 많이 사용합니다.

### 일반적인 시나리오

1. 유저가 "오래 걸리는 작업"을 요청
2. 요청을 받은 View에서 "오래 걸리는 작업"을 혼자서 실행하고, 실행이 완료되어야만 유저에게 응답을 줄 수 있습니다.
3. 실제 처리에 30초가 걸린다면, 유저는 30초 동안 모래시계만 한없이 바라보고 있어야만 합니다.
4. 기다리다 지쳐버린 유저는 새로고침을 누르고, 다시 작업을 요청 => 작업이 중복 요청되며, 악순환이 반복되는 거죠.

이때 Celery가 여러분을 구원해드립니다.

### 파이썬에서 사용할 수 있는 Task Queue 라이브러리

파이썬에서 사용할 수 있는 "Asynchronous Tasks Queue" 라이브러리가 다수 있지만, **Celery**가 가장 유연하고 막강하며, 대형 시스템에도 적합합니다.

#### [어썸 파이썬 리스트 - Queue](https://github.com/vinta/awesome-python#queue)

+ [**celery**](http://www.celeryproject.org/) - An asynchronous task queue/job queue based on distributed message passing.
+ [huey](https://github.com/coleifer/huey) - Little multi-threaded task queue.
+ [mrq](https://github.com/pricingassistant/mrq) - Mr. Queue - A distributed worker task queue in Python using Redis & gevent.
+ [rq](http://python-rq.org/) - Simple job queues for Python.
+ [simpleq](https://github.com/rdegges/simpleq) - A simple, infinitely scalable, Amazon SQS based queue.

### Celery를 통한 개선

1. [클라이언트] 유저가 "오래 걸리는 작업"을 요청
1. [뷰] 요청을 받은 View에서는, **Broker**를 특정 작업 실행을 위임하고 Task ID를 발급받음.
1. [뷰] 새로운 Task ID를 발급받은 View는 유저에게 해당 Task ID와 함께, 작업이 수행 중임을 알려줌. => 빠른 응답
1. [Broker] 놀고있는 Worker가 생기면 Broker로부터 작업을 받아서 실행하고,
    - 실행결과를 Broker에 저장
    - 이메일이나 푸쉬를 통한 알림

## 개발환경 셋업

### Broker 설치

[공식문서](http://docs.celeryproject.org/en/latest/getting-started/brokers/)

+ **RabbitMQ** : 실서비스에서는 무조건 이거 쓰세요. Celery는 RabbitMQ와 같이 쓸려고 만들어졌음.
    - 윈도우에서의 설치 : [공식 설치가이드](https://www.rabbitmq.com/install-windows.html)
        - 설치하면, RabbitMQ 이름으로 서비스가 등록이 되고, "자동 시작" 세팅이 된다고 합니다. (아직 확인 못 해봤습니다.)
    - 맥에서의 설치 : [공식 설치가이드](https://www.rabbitmq.com/install-standalone-mac.html)
        - homebrew에서도 팩키지를 지원 : `쉘> brew install rabbitmq`
        - 설치 후에, 터미널에서 `쉘> rabbitmq-server` 명령으로 실행하실 수 있습니다. rabbitmq-server 명령이 설치된 경로를 모르시는 분은 다음 명령으로 확인하실 수 있습니다. `쉘> find / -name rabbitmq-server 2> /dev/null`
+ Redis : 지원은 되나, 안정적이진 않습니다.
+ Amazon SQS : 지원은 되나, 안정적이진 않습니다.

아래 내용을 진행하기 위해, 필히 Broker 설치/구동 중이여야 합니다.

맥의 터미널에서 rabbitmq-server 명령을 수행하면, 다음과 같은 출력이 되며, 실행 중 상태입니다.

```
쉘> rabbitmq-server

              RabbitMQ 3.6.12. Copyright (C) 2007-2017 Pivotal Software, Inc.
  ##  ##      Licensed under the MPL.  See http://www.rabbitmq.com/
  ##  ##
  ##########  Logs: /usr/local/var/log/rabbitmq/rabbit@localhost.log
  ######  ##        /usr/local/var/log/rabbitmq/rabbit@localhost-sasl.log
  ##########
              Starting broker...
 completed with 9 plugins.

(실행 중 ...)
```

실행 중지를 하실려면, `Ctrl-C` 입력을 하세요.

### 설치할 파이썬 팩키지

```sh
pip3 install celery  # 2017년 9월 기준, 최신버전 4.1.0
```

이때, `celery` 명령도 설치됩니다. 이를 통해 여러 Celery명령을 수행할 수 있습니다.

Tip: `django-celery` 라이브러리는 더 이상 사용되지 않습니다. `celery` 팩키지에서 장고를 직접적으로 지원해주고 있습니다.

## 장고 프로젝트에 최소한의 Celery 셋업하기

참고 공식문서 : [First steps with Django](http://docs.celeryproject.org/en/latest/django/first-steps-with-django.html)

프로젝트 생성 및 이동

```
쉘> django-admin startproject djpj
쉘> cd djpj
```

현재 프로젝트 파일 현황

```
+ manage.py
+ djpj
    - __init__.py
    - settings.py
    - urls.py
    - wsgi.py
```

현재 `djpj/settings.py`에는 Celery관련 세팅을 전혀 하지 않고, 디폴트 설정을 그대로 쓰겠습니다. 디폴트 Broker는 **RabbitMQ**이므로, 필히 구동 중이어야 합니다.

### 최소한의 Celery 세팅

`djpj/celery.py` 파일 생성하고, celery를 통해 수행할 함수로서 mysum을 구현합니다.

```python
from celery import Celery

# Celery Application을 생성합니다. 생성된 이름은 다른 이름이어도 됩니다.
# Celery Application을 다수 만들수도 있으나, 대개 1개만 만듭니다.
app = Celery()

@app.task
def mysum(x, y):
    print('{} + {} = {}'.format(x, y, x + y))
    return x + y
```

`djpj/__init__.py` 파일 수정

```python
from .celery import app
```

### 작업 수행을 지시받으면, 작업을 수행할 Worker 대기 시키기

celery 명령에서는 다음 1가지 인자만 지정토록 하겠습니다.

1. -A (혹은 --app) : celery 애플리케이션이 있는 파이썬 팩키지를 지정합니다. 현재 `djpj/__init__.py` 에 임포트되어있으므로, **djpj** 를 지정하시면 됩니다.

아래 명령을 `manage.py` 파일이 있는 장고 프로젝트 루트 경로에서 실행해주세요. 계속 실행 중이여야 합니다. `--detach` 옵션을 지정하면 백그라운드로 구동됩니다.

```
쉘> celery worker -A djpj

celery@nomadekr.local v4.1.0 (latentcall)

Darwin-17.0.0-x86_64-i386-64bit 2017-10-11 23:27:37

[config]
.> app:         __main__:0x10c922710
.> transport:   amqp://guest:**@localhost:5672//
.> results:     disabled://
.> concurrency: 8 (prefork)
.> task events: OFF (enable -E to monitor tasks in this worker)

[queues]
.> celery           exchange=celery(direct) key=celery

(실행 중입니다. 끄시면 안 됩니다.)
```

위와 같이 디폴트 설정으로 worker가 실행이 되었습니다. 이제 Task 실행 지시를 받으면, 여유 Worker를 통해 Task가 실행이 됩니다.

+ transport : Broker를 뜻합니다. 디폴트 설정으로 로컬의 RabbitMQ 설정입니다.
+ results : 이는 실행된 Task의 리턴값을 저장하는 설정입니다. 디폴트 설정으로 꺼져있습니다. Task에서 리턴하더라도, 그 값은 저장되지 않고 유실됩니다.
+ concurrency : 큐에서 생성하는 자식 프로세스의 갯수입니다. 디폴트 설정으로 CPU 코어갯수가 설정됩니다.
+ task events : Task 이벤트를 통해 Task 현황을 조회할 수 있습니다. [Flower](http://flower.readthedocs.io/en/latest/)와 같은 웹기반 모니터링 툴을 써볼 수도 있겠습니다.

Worker에는 큐를 통해 Task가 전달됩니다. 이러한 큐는 여럿 만들 수 있습니다. 큐 별로 처리속도나 처리할 Worker를 다르게 설정하실 수도 있습니다. 현재는 `celery`라는 이름의 큐가 1개만 생성이 되어있네요.

### Celery Task인 mysum을 수행해봅시다.

장고 쉘을 시작하고, Task 함수를 임포트해봅시다.

```python
쉘> python3 manage.py shell

>>> from djpj.celery import mysum  # Task 함수를 임포트합니다.
```

이렇게 호출하면, Celery Task로서 실행하는 것이 아니라, 단순히 파이썬 함수를 호출한 것 뿐입니다.

```python
>>> mysum(1, 2)
1 + 2 = 3       # mysum 함수내에서 print된 문자열
3               # mysum(1, 2)의 리턴값
```

모든 Celery Task에는 delay함수가 지원됩니다. 필히 `.delay()`함수를 통해 호출해야합니다. 다음과 같이 호출할 수 있습니다.

```python
>>> result = mysum.delay(1, 2)

>>> result
<AsyncResult: 7aa3f2b1-539a-487c-8242-f3abee375f5e>
```

그런데, 리턴값이 `3`이 아니라, `AsyncResult` 타입의 객체입니다. Celery Task는 디폴트로 비동기로 수행이 됩니다. 그래서 리턴값을 나중에 조회할 수 있도록 `task id`를 부여받습니다. 그 값이 위의 `7aa3f2b1-539a-487c-8242-f3abee375f5e`이며, `result.id` 로도 조회하실 수 있습니다.

`result.get()`을 통해 리턴값을 조회할 수 있습니다. 그런데 현재 Celery설정에는 `result backend` 설정이 없으므로, 다음과 같은 예외가 발생합니다.

```python
>>> result.get()

(중략)
NotImplementedError: No result backend is configured.
Please see the documentation for more information.
```

`mysum.delay(1, 2)` 실행은 실제로 Worker에서 수행이 됩니다. `1 + 2 = 3` 문자열은 worker를 실행한 터미널을 보시면, 다음과 같은 오류가 찍혀있음을 확인하실 수 있습니다.

```
[2017-10-11 23:52:38,451: WARNING/ForkPoolWorker-1] 1 + 2 = 3
```

### Celery Task의 리턴값인 Result 를 저장/획득해봅시다.

Celery에서 지원하는 Result Backend 목록 ([공식문서](http://docs.celeryproject.org/en/latest/userguide/configuration.html#std:setting-result_backend))에 보시면, 다양한 Result Backend가 지원되고 있습니다. 이 중에 데이터베이스를 Result Backend로 써보겠습니다.

`djpj/settings.py` 소스코드에 다음 코드를 1줄 추가해주세요. DB엔진으로 sqlite3를 지정했으므로, 별도로 데이터베이스 서버를 구축하실 필요는 없습니다.

```python
CELERY_RESULT_BACKEND = 'db+sqlite:///results.sqlite'
```

이제 `djpj/celery.py` 파일을 수정하여, `djpj/settings.py` 내 `CELERY_` 로 시작하는 각종 설정을 로딩토록 해야합니다.

```python
import os
from celery import Celery

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'djpj.settings')  # DJANGO_SETTINGS_MODULE 환경변수 미정의시에 사용할 디폴트값을 설정합니다.

app = Celery()
app.config_from_object('django.conf:settings', namespace='CELERY')  # settings 내에서 로딩할 설정의 PREFIX

@app.task
def mysum(x, y):
    print('{} + {} = {}'.format(x, y, x + y))
    return x + y
```

2개의 코드가 추가되었습니다.

1. `app.config_from_object(...)` : 지정 `DJANGO_SETTINGS_MODULE`에 지정된 settings 로부터 지정 PREFIX의 설정을 로딩합니다.
2. 위 `app.config_from_object(...)`에서 읽어들일 settings의 위치는 `DJANGO_SETTINGS_MODULE` 환경변수를 통해 지정해야만 합니다. 이 환경변수에 대한 보다 자세한 내용은 [AskDjango VOD [장고 서비스 배포하기] 배포준비 (공통) - DJANGO\_SETTINGS\_MODULE](https://nomade.kr/vod/deploy/111/) 에피소드를 참고하세요.

장고 코드가 변경되었으므로, Celery Worker 와 장고쉘을 재시작해주세요. 재시작하지 않으면 변경된 설정이 반영되지 않습니다.

장고쉘을 통해, 다음과 같이 실행해볼 수 있겠습니다.

```python
쉘> python3 manage.py shell

>>> from djpj.celery import mysum

>>> result = mysum.delay(1, 2)

>>> result
>>> <AsyncResult: cf1f54c0-153f-4272-b729-e56b29a2f41c>

>>> result.get()
3
```

Celery Worker에는 다음과 같이 로그가 잘 찍혀있네요. :)

```
[2017-10-11 15:20:51,648: WARNING/ForkPoolWorker-5] 1 + 2 = 3
```

이와 같이 장고 프로젝트에서의 Celery를 간략하게나마 살펴봤습니다.

Celery 관련하여 다른 내용도 천천히 정리해보겠습니다.

감사합니다.

AskDjango 이진석 드림


# TODO

+ 장고 앱 별로 Celery Task 정의하기
+ Celery에서의 직렬화
+ Celery에서 Task 수행결과 (Result) 받기
+ 실서비스에서의 Celery 세팅
+ 각종 celery 명령들
+ celery 모니터링
+ ...

## 참고 문서

+ [First steps with Celery](http://docs.celeryproject.org/en/latest/getting-started/first-steps-with-celery.html)
+ [First steps with Django](http://docs.celeryproject.org/en/latest/django/first-steps-with-django.html)